### Self-driving Cab Example

In [21]:
import gym

In [22]:
env = gym.make('Taxi-v3').env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [23]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [24]:
state = env.encode(3, 1, 2, 0) #our state
print(f'State: {state}')
env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [25]:
env.P[0]

{0: [(1.0, 100, -1, False)],
 1: [(1.0, 0, -1, False)],
 2: [(1.0, 20, -1, False)],
 3: [(1.0, 0, -1, False)],
 4: [(1.0, 16, -1, False)],
 5: [(1.0, 0, -10, False)]}

In [26]:
#Brute force
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))


Timesteps taken: 1895
Penalties incurred: 599


In [27]:
#Q-learning
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [28]:
q_table.shape

(500, 6)

In [29]:
%%time

import random
from IPython.display import clear_output

#hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

#for plotting metrics
all_epochs = []
all_penalties = []

#for i in range(1, 100001):
for i in range(3):
    state = env.reset()
    print(f'State:{state}')
    
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        print(action)
            
        next_state, reward, done, info = env.step(action) 
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
    if i%10000 == 0:
        print(f"Episode: {i}")

print("Training finished.\n")


Episode: 0
State:33
0
0
5
0
3
0
1
1
1
1
1
2
3
0
0
0
0
0
1
1
1
1
2
4
5
0
2
3
2
4
5
0
2
0
0
0
1
1
1
0
2
3
3
3
2
4
0
2
2
3
4
5
0
4
0
0
2
2
4
3
3
5
1
1
4
1
1
1
2
4
3
1
3
4
5
0
3
4
5
0
0
2
3
4
5
0
2
3
4
5
0
4
3
1
5
1
0
2
3
0
2
0
1
1
4
3
5
1
1
1
2
0
2
3
2
0
2
4
5
0
4
5
0
5
0
1
1
1
1
0
2
0
0
0
0
1
1
1
1
0
2
4
0
5
0
0
0
1
1
1
1
0
2
3
3
3
4
5
0
2
1
4
5
0
2
2
3
4
5
0
2
2
3
3
4
5
2
4
5
0
2
3
2
4
5
1
1
4
0
2
3
0
3
4
0
1
0
2
0
1
0
2
3
3
4
5
5
0
1
1
2
5
1
5
0
2
3
3
3
3
3
2
0
2
2
3
3
0
2
2
3
5
3
0
1
1
2
0
0
0
1
1
1
1
1
2
1
2
1
2
3
3
3
4
5
0
2
1
4
0
2
1
4
5
0
2
3
2
1
1
2
3
5
1
2
3
3
1
2
2
0
0
1
2
3
3
3
1
3
0
0
2
3
3
1
1
3
3
0
3
0
1
1
1
3
2
1
2
0
2
3
3
3
2
0
3
3
2
0
2
5
2
3
3
0
2
2
3
3
0
1
1
2
1
2
0
0
3
0
2
0
5
1
0
2
3
3
4
0
1
1
0
2
1
0
2
3
3
5
4
0
3
2
4
5
0
1
4
2
5
0
2
3
5
0
1
2
1
0
2
3
1
1
1
0
3
3
1
1
3
2
0
0
2
2
2
0
0
0
2
3
1
3
0
2
1
1
3
0
2
2
3
1
1
1
1
3
0
0
0
0
0
1
1
4
2
2
1
1
1
2
3
1
2
0
3
2
0
3
3
3
1
1
3
2
1
2
0
2
3
0
0
2
3
1
1
1
0
2
0
3
3
2
0
2
2
3
3
0
2
2
3
2
1
0
2
0
3
3
0
1
1
2
1
2
3
3
2
0
0


In [30]:
q_table[328]

array([0., 0., 0., 0., 0., 0.])

In [31]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

KeyboardInterrupt: 